In [ ]:
import pandas as pd
from openai import OpenAI
import pickle
import time
from tqdm.notebook import tqdm

In [ ]:
df_test = pd.read_csv('cook_en_test.csv')
df_train = pd.read_csv('cook_en_train.csv')

In [ ]:
client = OpenAI(api_key="")

In [ ]:
results = []

for t in tqdm(range(len(df_test))):  # Loop over the range of idioms
    i = df_test['submission'].iloc[t]  # Access each idiom directly using the loop index

    literals = []
    for m in range(3):
        if len(df_train[(df_train['category'] == 'literal') & (df_train['idiom'] == df_test['idiom'].iloc[t])]) == 0:
            text = df_train[(df_train['category'] == 'literal')].sample(1)['submission'].iloc[0]
        else:
            text = df_train[(df_train['category'] == 'literal') & (df_train['idiom'] == df_test['idiom'].iloc[t])].sample(1)['submission'].iloc[0]
        literals.append(text)

    figuratives = []
    for m in range(3):
        if len(df_train[(df_train['category'] == 'figurative') & (df_train['idiom'] == df_test['idiom'].iloc[t])]) == 0:
            text = df_train[(df_train['category'] == 'figurative')].sample(1)['submission'].iloc[0]
        else:
            text = df_train[(df_train['category'] == 'figurative') & (df_train['idiom'] == df_test['idiom'].iloc[t])].sample(1)['submission'].iloc[0]

        figuratives.append(text)

    completion = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
    {"role": "system", "content": 
        "Your task is to be a language model that speaks English. When responding, follow the given instructions precisely and do not add anything else."
    },
    {"role": "user", "content": 
        "I want you to determine whether the following sentence contains an idiom. If the sentence contains an idiom, write '1'. If it does not, write '0'. Only respond with 0 or 1, do not write anything else. Sentence:\n"
        + literals[0] + ": 0\n" + literals[1] + ": 0\n" + literals[2] + ": 0\n"
        + figuratives[0] + ": 1\n" + figuratives[1] + ": 1\n" + figuratives[2] + ": 1\n"
        + i + ": ?"
    }
    ]
    )
    print(completion.choices[0].message.content)

    while completion.choices[0].message.content == "?" or len(completion.choices[0].message.content) > 1:
        completion = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
    {"role": "system", "content":
        "Your task is to be a language model that speaks English. When responding, follow the given instructions precisely and do not add anything else."
    },
    {"role": "user", "content":
        "I want you to determine whether the following sentence contains an idiom. If the sentence contains an idiom, write '1'. If it does not, write '0'. Only respond with 0 or 1, do not write anything else. Sentence:\n"
        + literals[0] + ": 0\n" + literals[1] + ": 0\n" + literals[2] + ": 0\n"
        + figuratives[0] + ": 1\n" + figuratives[1] + ": 1\n" + figuratives[2] + ": 1\n"
        + i + ": ?"
    }
    ]
    )

    results.append(completion)  # Append the result to the list

In [ ]:
len(df_test["category"].tolist())

In [ ]:
len(df_test)


In [ ]:
with open('gpt-4-0125-preview_en.pkl', 'wb') as f:
      pickle.dump(results, f)

In [ ]:
labels = df_test["category"].tolist()

In [ ]:
df = pd.DataFrame([i.choices[0].message.content for i in results])
df.to_csv('results_gpt-4-turbo_en.csv', index=False)

In [ ]:
# read df back

In [ ]:
df = pd.read_csv('results_gpt-4-turbo_en.csv')

In [ ]:
for i in [i for i, x in enumerate(df[0].tolist()) if x == "1\n"]:
    df[0][i] = "1"

In [ ]:
df.iloc[:, 0].tolist()

In [ ]:
gpt_labels = [int(i) for i in df.iloc[:, 0].tolist()]


In [ ]:
human_labels = [1 if i == "figurative" else 0 for i in labels]

In [ ]:
gpt_labels

In [ ]:
human_labels

In [ ]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(human_labels, gpt_labels)

print(report)


In [ ]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(human_labels, gpt_labels, digits=4)

# Write report to a txt file
with open('classification_report_gpt-4-0125-preview_en.txt', 'w') as f:
    f.write(report)
print(report)
